In [1]:
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ML Pre processing
from surprise.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Hyperparameter tuning
from surprise.model_selection import GridSearchCV

C:\Users\Acer\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Acer\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Acer\anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msn

In [8]:
df_ratings = pd.read_csv('modified_rating_dataSet.csv')

In [9]:
df_ratings.head(2)

,userID,movieID,rating
0,75,3,1.0
1,75,32,4.5


In [10]:
reader = Reader()

In [11]:
data = Dataset.load_from_df(df_ratings[['userID','movieID','rating']], reader)

In [15]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse') 

,test_rmse,fit_time,test_time
Algorithm,,,
SVD,0.781084,53.535068,4.417511


In [13]:
algo = SVD()

In [14]:
results = cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=True)

Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7810  0.7825  0.7808  0.7815  0.0008  
Fit time          62.98   64.25   64.74   63.99   0.74    
Test time         4.73    5.85    4.04    4.87    0.74    


In [29]:
param_grid = {'n_epochs': [7,9,12,15,20,30,35,40], 'lr_all': [0.002,0.003,0.004,0.005,0.007,0.009],
              'reg_all': [0.2,0.4,0.5, 0.6,0.7,0.8,0.9]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

KeyboardInterrupt: 

In [22]:
algo = gs.best_estimator['rmse']

In [23]:
trainset, testset = train_test_split(data, test_size=0.01)

In [24]:
algo.fit(trainset)

In [25]:
predictions = algo.test(testset) 

In [ ]:
results.rmse(predictions)

In [27]:
accuracy.mae(predictions)

MAE:  0.6461


0.6460653203274601

In [28]:
algo.predict(1, 5107) 

Prediction(uid=1, iid=5107, r_ui=None, est=3.169762116226633, details={'was_impossible': False})